In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf


In [10]:
jurosmedmensal= pd.read_csv('http://api.bcb.gov.br/dados/serie/bcdata.sgs.25433/dados?formato=csv',
                  sep = ';', encoding = 'utf-8', decimal = ',')
txinadimplencia = pd.read_csv  ('http://api.bcb.gov.br/dados/serie/bcdata.sgs.21082/dados?formato=csv',
                                sep = ';', encoding = 'utf-8', decimal = ',')


In [7]:
jurosmedmensal.head()

,data,valor
0,01/03/2011,2.05
1,01/04/2011,2.09
2,01/05/2011,2.08
3,01/06/2011,2.07
4,01/07/2011,2.08


In [11]:
txinadimplencia.head()

,data,valor
0,01/03/2011,3.18
1,01/04/2011,3.25
2,01/05/2011,3.37
3,01/06/2011,3.33
4,01/07/2011,3.42
